In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

# DATA PREPROCESSING

In [ ]:
df.isnull().sum()

In [ ]:
#fill the missing values for numerical terms -- mean
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mean())
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mean())
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mean())

In [ ]:
#fill na values for categorical terms -- mode
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])

In [ ]:
df.isnull().sum()

# Explotatory Analysis

In [ ]:
# categorical visualisation
plt.figure(figsize = (9,5))
sns.countplot(df['Gender'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Married'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Dependents'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Self_Employed'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Education'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Property_Area'])

In [ ]:
plt.figure(figsize = (9,5))
sns.countplot(df['Loan_Status'])

In [ ]:
# numerical visualization
plt.figure(figsize = (12,15))
sns.displot(df['ApplicantIncome'])

In [ ]:
#apply for transformation to the attribute
df['ApplicantIncomeLog'] = np.log(df['ApplicantIncome'])

In [ ]:
plt.figure(figsize = (12,15))
sns.displot(df['ApplicantIncomeLog'])

In [ ]:

plt.figure(figsize = (16,19))
sns.displot(df['CoapplicantIncome'])

In [ ]:
df['CoapplicantIncomeLog']= np.log(df['CoapplicantIncome']*100)



In [ ]:
plt.figure(figsize = (12,15))
sns.displot(df['LoanAmount'])

In [ ]:
df['LoanAmountLog'] = np.log(df['LoanAmount'])

In [ ]:
plt.figure(figsize = (12,15))
sns.displot(df['LoanAmountLog'])

In [ ]:
sns.displot(df['Loan_Amount_Term'])

In [ ]:
df['Loan_Amount_TermLog']= np.log(df['Loan_Amount_Term'])

In [ ]:
sns.displot(df['Loan_Amount_TermLog'])

In [ ]:
sns.displot(df['Credit_History'])

# creating new attributes

In [ ]:
#total income 
df['Total_income'] = df['ApplicantIncome']+ df['CoapplicantIncome']
df.head()

In [ ]:
df['Total_incomeLog']= np.log(df['Total_income'])

In [ ]:
df.head()

# Correlation matrix

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize = (15,9))
sns.heatmap(df.corr(), annot = True)

In [ ]:
#drop unnecessary columns
cols = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term', 'Total_income', 'Loan_ID', 'CoapplicantIncomeLog']
df = df.drop(columns = cols, axis =1)
df.head()

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ['Gender', 'Married','Education','Self_Employed','Property_Area','Loan_Status', 'Dependents']
le = LabelEncoder()
for col in cols:
    df[col]= le.fit_transform(df[col])

In [ ]:
df.head()

# Train Test Split

In [ ]:
x = df.drop(columns = ['Loan_Status'], axis = 1)
y = df['Loan_Status']


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25, random_state=51)

# Model Selection

In [ ]:
from sklearn.model_selection import cross_val_score
# define classify function
def classify(model,x,y):
    x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25, random_state=51)
    model.fit(x_train,y_train)
    print('Accuracy is ', model.score(x_test,y_test)*100)
    #cross_validation basically used for better validation of model
    score = cross_val_score(model,x,y,cv=5)
    print('cross_validation ', np.mean(score)*100)
    

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model,x,y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classify(model,x,y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
model = RandomForestClassifier()
classify(model,x,y)

In [ ]:
model = ExtraTreesClassifier()
classify(model,x,y)

# Hyper parameter tuning

In [ ]:
model = RandomForestClassifier(n_estimators=50,min_samples_split=18, max_depth=7, max_features=1)
classify(model,x,y)

# Confusion Matrix

In [ ]:
model =RandomForestClassifier()
model.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
sns.heatmap(cm,annot=True)